In [11]:
# Import all the required libraries 

from glob import glob
import mne
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from scipy import stats
from scipy import signal
from scipy import fftpack
import pywt
import os,sys
from scipy.signal import coherence
from neurodsp.aperiodic import compute_irasa, fit_irasa

In [12]:
all_files = glob('F:\Alabama_Internship\EEG\Deepesh_Original_EEG_Data\*.edf')
print(len(all_files))
print(all_files[0])

80
F:\Alabama_Internship\EEG\Deepesh_Original_EEG_Data\H10_EC_post.edf


In [13]:
EC_pre = [i for i in all_files if 'EC_pre' in i.split("\\")[4]]
EC_post = [i for i in all_files if 'EC_post' in i.split("\\")[4]]

EC_pre.sort()
EC_post.sort()

print(EC_pre[19], EC_post[19])

F:\Alabama_Internship\EEG\Deepesh_Original_EEG_Data\H9_EC_pre.edf F:\Alabama_Internship\EEG\Deepesh_Original_EEG_Data\H9_EC_post.edf


In [14]:
%%capture
sf = 1000 # Sampling frequency

alpha = [8, 13]
beta = [13, 30]
delta = [1.25, 4]
theta = [4, 8]
gamma = [30, 48]

fractal_pre_all = []

for file in EC_pre:
    raw = mne.io.read_raw_edf(file, preload=True, exclude=['EEG VREF'])
    raw = raw.crop(tmin=10, tmax=300)
    raw = (raw.copy()).filter(delta[0], gamma[1])

    patient = []

    for i in range(64):
        freqs, psd_aperiodic, psd_periodic = compute_irasa(raw.get_data()[i], sf, f_range=(delta[0], gamma[1]), thresh=1)
        intercept, fit_sl = fit_irasa(freqs, psd_aperiodic)
        patient.append(fit_sl)

    fractal_pre_all.append(patient)


In [15]:
%%capture
sf = 1000 # Sampling frequency

alpha = [8, 13]
beta = [13, 30]
delta = [1.25, 4]
theta = [4, 8]
gamma = [30, 48]

fractal_post_all = []

for file in EC_post:
    raw = mne.io.read_raw_edf(file, preload=True, exclude=['EEG VREF'])
    raw = raw.crop(tmin=10, tmax=300)
    raw = (raw.copy()).filter(delta[0], gamma[1])

    patient = []

    for i in range(64):
        freqs, psd_aperiodic, psd_periodic = compute_irasa(raw.get_data()[i], sf, f_range=(delta[0], gamma[1]), thresh=1)
        intercept, fit_sl = fit_irasa(freqs, psd_aperiodic)
        patient.append(fit_sl)

    fractal_post_all.append(patient)

Looking into the relationship between all of the channels instead of combining all channels into one

In [16]:
cols = []
bands = ['channel']

for i in range(1):
    for j in range(64):
        cols.append(bands[i] + str(j + 1))

cols.append('target')

In [17]:
all_combined_pre = []
all_combined_post = []

for i in range(20):
    all_combined_pre.append(fractal_pre_all[i] + [1])
    all_combined_post.append(fractal_post_all[i] + [0])

In [19]:
all_combined = all_combined_pre + all_combined_post
df = pd.DataFrame(all_combined, columns = cols, dtype = float)
df.to_excel('Dataset_fractal_full_spectrum.xlsx')

Computing pairwise T plots for all powers

In [20]:
fractal_pre_all_t = np.array(fractal_pre_all).T.tolist()
fractal_post_all_t = np.array(fractal_post_all).T.tolist()

def compute_ttest(pre, post):
    results = []
    for i in range(64):
        results.append(stats.ttest_rel(pre[i], post[i]))
        if(results[i].pvalue <= 0.05):
            print(i + 1)

print("CHANNELS")
compute_ttest(fractal_pre_all_t, fractal_post_all_t)

CHANNELS
49
